# Assignment 2: Complete Solutions

This notebook contains solutions to all problems in Assignment 2.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from math import floor, ceil

# Set up plotting
plt.rcParams['figure.figsize'] = (10, 6)

## Question 1: Triangle-Free Graphs

Maximum edges: $m \leq \lfloor n^2/4 \rfloor$.

### (a) Examples for $n \in \{2,3,4,5,6\}$

Complete bipartite graphs $K_{\lfloor n/2 \rfloor, \lceil n/2 \rceil}$ achieve the bound.

**Common property:** All are complete bipartite graphs with balanced partitions.

In [ ]:
def draw_complete_bipartite(n1, n2, title=""):
    """Draw a complete bipartite graph K_{n1,n2}"""
    G = nx.complete_bipartite_graph(n1, n2)
    pos = nx.bipartite_layout(G, list(range(n1)))
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(G, pos, nodelist=list(range(n1)), node_color='lightcoral', node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=list(range(n1, n1+n2)), node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(G, pos, alpha=0.6)
    nx.draw_networkx_labels(G, pos)
    plt.title(title if title else f"$K_{{{n1},{n2}}}$: {n1*n2} edges")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Draw examples for n = 2, 3, 4, 5, 6
for n in [2, 3, 4, 5, 6]:
    n1 = floor(n/2)
    n2 = ceil(n/2)
    max_edges = floor(n**2/4)
    draw_complete_bipartite(n1, n2, f"n={n}: $K_{{{n1},{n2}}}$ with {max_edges} edges (max: $\lfloor {n}^2/4 \rfloor = {max_edges}$)")

### (b) General Construction

For arbitrary $n$: $G = K_{\lfloor n/2 \rfloor, \lceil n/2 \rceil}$.

Edges: $\lfloor n/2 \rfloor \cdot \lceil n/2 \rceil = \lfloor n^2/4 \rfloor$ (optimal).

## Question 2: Bi-graphical Sequences

### (a) $S_1 = \langle 6, 5, 5, 5, 3, 2, 1, 1 \rangle$, $S_2 = \langle 5, 5, 4, 3, 2 \rangle$

Since $a_1 = 6 > |S_2| = 5$, the condition $a_1 \le s$ is violated. **Not bi-graphical.**

In [ ]:
def is_bigraphical(S1, S2, verbose=False):
    """
    Determine if the pair (S1, S2) is bi-graphical using the stated proposition.
    """
    S1 = sorted(S1, reverse=True)
    S2 = sorted(S2, reverse=True)
    r, s = len(S1), len(S2)
    
    if sum(S1) != sum(S2):
        if verbose:
            print(f"  Sum mismatch: sum(S1)={sum(S1)}, sum(S2)={sum(S2)}")
        return False
    if any(d < 0 for d in S1 + S2):
        if verbose:
            print("  Negative degree")
        return False
    
    if r == 0 and s == 0:
        return True
    if r == 0 or s == 0:
        if verbose:
            print("  One side empty, other non-empty")
        return False
    
    a1, b1 = S1[0], S2[0]
    # Base case: one left vertex => right must be a1 ones and (s-a1) zeros
    if r == 1:
        return sum(S2) == a1 and sorted(S2, reverse=True) == [1] * a1 + [0] * (s - a1)
    if s == 1:
        return sum(S1) == b1 and sorted(S1, reverse=True) == [1] * b1 + [0] * (r - b1)
    
    if a1 > s:
        if verbose:
            print(f"  a1={a1} > s={s}")
        return False
    if b1 > r:
        if verbose:
            print(f"  b1={b1} > r={r}")
        return False
    if a1 == 0:
        return is_bigraphical(S1[1:], S2, verbose)
    if b1 == 0:
        return is_bigraphical(S1, S2[1:], verbose)
    
    # Reduction: (a2,...,ar) and (b1-1,...,b_{a1}-1, b_{a1+1},...,bs)
    S1_new = S1[1:]
    S2_new = [S2[i] - 1 if i < a1 else S2[i] for i in range(s)]
    S2_new = sorted(S2_new, reverse=True)
    if any(d < 0 for d in S2_new):
        if verbose:
            print("  Reduction gives negative degree")
        return False
    
    if verbose:
        print(f"  Reduce: S1'={S1_new}, S2'={S2_new}")
    return is_bigraphical(S1_new, S2_new, verbose)

# Part (a)
S1_a = [6, 5, 5, 5, 3, 2, 1, 1]
S2_a = [5, 5, 4, 3, 2]
print("Part (a):")
print(f"  sum(S1) = {sum(S1_a)}, sum(S2) = {sum(S2_a)}")
ans_a = is_bigraphical(S1_a, S2_a, verbose=True)
print(f"  Bi-graphical? {ans_a}")
print("  Answer: No. The sequences are not bi-graphical (a1=6 > |S2|=5).")

### (b) $S_1 = \langle 8, 6, 4, 4, 4, 4, 4 \rangle$, $S_2 = \langle 6, 5, 4, 4, 4, 4, 3, 3, 1 \rangle$

Reduction procedure succeeds. **Bi-graphical.**

In [ ]:
# Part (b)
S1_b = [8, 6, 4, 4, 4, 4, 4]
S2_b = [6, 5, 4, 4, 4, 4, 3, 3, 1]
print("Part (b):")
print(f"  sum(S1) = {sum(S1_b)}, sum(S2) = {sum(S2_b)}")
ans_b = is_bigraphical(S1_b, S2_b, verbose=True)
print(f"  Bi-graphical? {ans_b}")

# Draw the bipartite graph
if ans_b:
    G = nx.bipartite.configuration_model(S1_b, S2_b)
    n1, n2 = len(S1_b), len(S2_b)
    S1_nodes = list(range(n1))
    S2_nodes = list(range(n1, n1 + n2))
    pos = nx.bipartite_layout(G, S1_nodes, align='vertical')
    for node in S1_nodes:
        pos[node] = (0, pos[node][1])
    for node in S2_nodes:
        pos[node] = (1, pos[node][1])
    plt.figure(figsize=(12, 8))
    nx.draw_networkx_nodes(G, pos, nodelist=S1_nodes, node_color='lightcoral', node_size=500, label='S1')
    nx.draw_networkx_nodes(G, pos, nodelist=S2_nodes, node_color='lightblue', node_size=500, label='S2')
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    labels = {i: f'u{i}\n({S1_b[i]})' for i in S1_nodes}
    labels.update({i: f'v{i-n1}\n({S2_b[i-n1]})' for i in S2_nodes})
    nx.draw_networkx_labels(G, pos, labels, font_size=9)
    plt.legend()
    plt.axis('off')
    plt.title("Part (b): A bipartite graph with the given degree sequences")
    plt.tight_layout()
    plt.show()

## Question 3: Connectivity

### (a) Graphs on 7 vertices with $\deg(v) \geq 3$

**Observation:** Cannot be disconnected. Each component needs $\geq 4$ vertices, but $4+4=8 > 7$.

In [ ]:
# Example graphs on 7 vertices with deg(v) >= 3
def draw_graph_7_vertices():
    # Graph 1: Complete graph K7 minus a perfect matching
    G1 = nx.complete_graph(7)
    # Remove edges: (0,1), (2,3), (4,5) to keep degree >= 3
    G1.remove_edges_from([(0,1), (2,3), (4,5)])
    
    # Graph 2: Another construction
    G2 = nx.complete_graph(7)
    G2.remove_edges_from([(0,2), (1,3), (4,6)])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    pos1 = nx.spring_layout(G1, seed=42)
    nx.draw_networkx_nodes(G1, pos1, ax=axes[0], node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(G1, pos1, ax=axes[0], alpha=0.6)
    nx.draw_networkx_labels(G1, pos1, ax=axes[0])
    axes[0].set_title(f"Graph 1: Connected, min degree = {min(dict(G1.degree()).values())}")
    axes[0].axis('off')
    
    pos2 = nx.spring_layout(G2, seed=43)
    nx.draw_networkx_nodes(G2, pos2, ax=axes[1], node_color='lightcoral', node_size=500)
    nx.draw_networkx_edges(G2, pos2, ax=axes[1], alpha=0.6)
    nx.draw_networkx_labels(G2, pos2, ax=axes[1])
    axes[1].set_title(f"Graph 2: Connected, min degree = {min(dict(G2.degree()).values())}")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Both graphs are connected: G1 is {nx.is_connected(G1)}, G2 is {nx.is_connected(G2)}")

draw_graph_7_vertices()

### (b) Graphs on 8 vertices with $\deg(v) \geq 4$

**Observation:** Must be connected. Each component needs $\geq 5$ vertices, but $5+5=10 > 8$.

In [ ]:
# Example graphs on 8 vertices with deg(v) >= 4
def draw_graph_8_vertices():
    # Graph 1: Complete graph K8 minus some edges
    G1 = nx.complete_graph(8)
    # Remove a few edges while keeping degree >= 4
    G1.remove_edges_from([(0,1), (2,3), (4,5)])
    
    # Graph 2: Another construction
    G2 = nx.complete_graph(8)
    G2.remove_edges_from([(0,2), (1,3), (4,6), (5,7)])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    pos1 = nx.spring_layout(G1, seed=42)
    nx.draw_networkx_nodes(G1, pos1, ax=axes[0], node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(G1, pos1, ax=axes[0], alpha=0.6)
    nx.draw_networkx_labels(G1, pos1, ax=axes[0])
    axes[0].set_title(f"Graph 1: Connected, min degree = {min(dict(G1.degree()).values())}")
    axes[0].axis('off')
    
    pos2 = nx.spring_layout(G2, seed=43)
    nx.draw_networkx_nodes(G2, pos2, ax=axes[1], node_color='lightcoral', node_size=500)
    nx.draw_networkx_edges(G2, pos2, ax=axes[1], alpha=0.6)
    nx.draw_networkx_labels(G2, pos2, ax=axes[1])
    axes[1].set_title(f"Graph 2: Connected, min degree = {min(dict(G2.degree()).values())}")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Both graphs are connected: G1 is {nx.is_connected(G1)}, G2 is {nx.is_connected(G2)}")

draw_graph_8_vertices()

### (c) Proof

**Theorem:** If $\deg(v) \geq \frac{n-1}{2}$ for all $v$, then $G$ is connected.

**Proof:** Suppose disconnected. Components containing $u$ and $v$ each have size $\geq \deg(u)+1 \geq \frac{n+1}{2}$. Then total vertices $\geq \frac{n+1}{2} + \frac{n+1}{2} = n+1 > n$, contradiction. $\square$

## Question 4: Graph Isomorphism

### Figure 2

Both: 7 vertices, 6 edges, degree sequence $\langle 3, 3, 2, 1, 1, 1, 1 \rangle$, trees. **Isomorphic.**

In [ ]:
# Figure 2: Two trees with same degree sequence
def draw_figure2():
    # Tree 1: Central vertex with branches
    G1 = nx.Graph()
    G1.add_edges_from([(0,1), (1,2), (1,3), (1,4), (2,5), (2,6)])
    
    # Tree 2: Different structure but same degree sequence
    G2 = nx.Graph()
    G2.add_edges_from([(0,1), (0,2), (0,3), (1,4), (2,5), (3,6)])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    pos1 = nx.spring_layout(G1, seed=42)
    nx.draw_networkx_nodes(G1, pos1, ax=axes[0], node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(G1, pos1, ax=axes[0], alpha=0.6)
    nx.draw_networkx_labels(G1, pos1, ax=axes[0])
    deg_seq1 = sorted([d for v, d in G1.degree()], reverse=True)
    axes[0].set_title(f"Tree 1: Degree sequence {deg_seq1}")
    axes[0].axis('off')
    
    pos2 = nx.spring_layout(G2, seed=43)
    nx.draw_networkx_nodes(G2, pos2, ax=axes[1], node_color='lightcoral', node_size=500)
    nx.draw_networkx_edges(G2, pos2, ax=axes[1], alpha=0.6)
    nx.draw_networkx_labels(G2, pos2, ax=axes[1])
    deg_seq2 = sorted([d for v, d in G2.degree()], reverse=True)
    axes[1].set_title(f"Tree 2: Degree sequence {deg_seq2}")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Degree sequences match: {deg_seq1 == deg_seq2}")
    print(f"Both are trees: {nx.is_tree(G1)} and {nx.is_tree(G2)}")
    print(f"Isomorphic: {nx.is_isomorphic(G1, G2)}")

draw_figure2()

### Figure 3

Left: $\langle 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2 \rangle$. Right: $\langle 5, 5, 5, 5, 3, 3, 3, 3, 2, 2, 2, 2 \rangle$. **Not isomorphic** (different degree sequences).

## Question 5: Complement of Paths

### (a) Complements of $P_n$ for $n = 2, 3, 4, 5, 6$

In [ ]:
def draw_path_and_complement(n):
    """Draw P_n and its complement"""
    P = nx.path_graph(n)
    P_complement = nx.complement(P)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    pos_P = nx.spring_layout(P, seed=42)
    nx.draw_networkx_nodes(P, pos_P, ax=axes[0], node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(P, pos_P, ax=axes[0], alpha=0.6, width=2)
    nx.draw_networkx_labels(P, pos_P, ax=axes[0])
    axes[0].set_title(f"$P_{n}$")
    axes[0].axis('off')
    
    pos_comp = nx.spring_layout(P_complement, seed=43)
    nx.draw_networkx_nodes(P_complement, pos_comp, ax=axes[1], node_color='lightcoral', node_size=500)
    nx.draw_networkx_edges(P_complement, pos_comp, ax=axes[1], alpha=0.6)
    nx.draw_networkx_labels(P_complement, pos_comp, ax=axes[1])
    connected = nx.is_connected(P_complement)
    axes[1].set_title(f"$\overline{{P_{n}}}$ (Connected: {connected})")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return connected

# Check for n = 2, 3, 4, 5, 6
results = {}
for n in range(2, 7):
    print(f"\n=== n = {n} ===")
    results[n] = draw_path_and_complement(n)

### Observations

From the examples:
- $\overline{P_2}$: Not connected (two isolated vertices)
- $\overline{P_3}$: Not connected (isolated vertex + edge)
- $\overline{P_4}$: Connected
- $\overline{P_5}$: Connected
- $\overline{P_6}$: Connected

**Pattern:** The complement of $P_n$ is connected for all $n \geq 4$.

**Pattern:** $\overline{P_n}$ is connected if and only if $n \geq 4$.

### (b) Conjecture and Proof

**Conjecture:** $\overline{P_n}$ is connected if and only if $n \geq 4$.

**Proof:** For $n \geq 4$, any two vertices in $\overline{P_n}$ are either adjacent or have a common neighbor (diameter $\leq 2$), so connected. For $n=2,3$, $\overline{P_n}$ is disconnected. $\square$

### (c) Generalization to Trees

**No.** Trees with high degree vertices or large diameter have complements with different connectivity properties than paths.

## Question 6: Graph Multiplication (Cartesian Product)

$G * H$: vertices $(u,a)$ where $u \in V(G)$, $a \in V(H)$; edges $((u,a),(v,b))$ if $u=v$ and $ab \in E(H)$, or $a=b$ and $uv \in E(G)$.

### (a) Draw $P_2 * K_3$ and $P_3 * K_3$

### (b) Edge-count formula: $m = n_1 m_2 + n_2 m_1$

In [ ]:
def cartesian_product(G: nx.Graph, H: nx.Graph) -> nx.Graph:
    """Cartesian product G * H implemented from the definition.

    Vertices: pairs (u, a) with u in V(G), a in V(H).
    Edges: ((u,a),(v,b)) if
      - u == v and (a,b) in E(H), OR
      - a == b and (u,v) in E(G).
    """
    K = nx.Graph()
    # Add all vertex pairs
    for u in G.nodes():
        for a in H.nodes():
            K.add_node((u, a))

    # Edges coming from H (fix u, vary a,b)
    for u in G.nodes():
        for a, b in H.edges():
            K.add_edge((u, a), (u, b))

    # Edges coming from G (fix a, vary u,v)
    for u, v in G.edges():
        for a in H.nodes():
            K.add_edge((u, a), (v, a))

    return K


def draw_product(G, H, G_name="G", H_name="H"):
    K = cartesian_product(G, H)
    pos = nx.spring_layout(K, seed=42)
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(K, pos, node_color="lightgreen", node_size=400)
    nx.draw_networkx_edges(K, pos, alpha=0.6)
    labels = {node: f"({node[0]},{node[1]})" for node in K.nodes()}
    nx.draw_networkx_labels(K, pos, labels, font_size=8)
    plt.title(f"Cartesian product {G_name} * {H_name}: |V|={K.number_of_nodes()}, |E|={K.number_of_edges()}")
    plt.axis("off")
    plt.tight_layout()
    plt.show()
    return K

# Part (a): draw P2 * K3 and P3 * K3
P2 = nx.path_graph(2)
P3 = nx.path_graph(3)
K3 = nx.complete_graph(3)

print("P2 * K3:")
K_P2K3 = draw_product(P2, K3, "P2", "K3")

print("P3 * K3:")
K_P3K3 = draw_product(P3, K3, "P3", "K3")


# Part (b): verify the edge-count formula m = n1*m2 + n2*m1

def edge_count_product(G, H):
    n1, m1 = G.number_of_nodes(), G.number_of_edges()
    n2, m2 = H.number_of_nodes(), H.number_of_edges()
    K = cartesian_product(G, H)
    m = K.number_of_edges()
    formula = n1 * m2 + n2 * m1
    print(f"G: n1={n1}, m1={m1}; H: n2={n2}, m2={m2}")
    print(f"G * H: |V|={K.number_of_nodes()}, |E|={m}")
    print(f"Formula n1*m2 + n2*m1 = {formula}")
    print(f"Matches formula? {m == formula}\n")

print("\nVerify edge-count formula on several examples:\n")
examples_G = [nx.path_graph(2), nx.path_graph(3), nx.cycle_graph(4)]
examples_H = [nx.complete_graph(3), nx.path_graph(4)]

for i, G in enumerate(examples_G, start=1):
    for j, H in enumerate(examples_H, start=1):
        print(f"Example G{i} * H{j}:")
        edge_count_product(G, H)

## Question 7: Bipartite Graphs with Bipartite Complements

### (a) Find a bipartite graph whose complement is also bipartite

**Example:** $K_{2,2}$. Its complement consists of two disjoint edges, which is bipartite.

In [ ]:
# Example: K_{2,2} and its complement
def draw_bipartite_with_bipartite_complement():
    # Create K_{2,2}
    G = nx.complete_bipartite_graph(2, 2)
    G_complement = nx.complement(G)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Draw original graph
    pos = nx.bipartite_layout(G, [0, 1])
    nx.draw_networkx_nodes(G, pos, nodelist=[0, 1], node_color='lightcoral', ax=axes[0], node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=[2, 3], node_color='lightblue', ax=axes[0], node_size=500)
    nx.draw_networkx_edges(G, pos, ax=axes[0], alpha=0.6, width=2)
    nx.draw_networkx_labels(G, pos, ax=axes[0])
    axes[0].set_title("$K_{2,2}$ (Bipartite)")
    axes[0].axis('off')
    
    # Draw complement
    pos_comp = nx.spring_layout(G_complement, seed=42)
    nx.draw_networkx_nodes(G_complement, pos_comp, ax=axes[1], node_color='lightgreen', node_size=500)
    nx.draw_networkx_edges(G_complement, pos_comp, ax=axes[1], alpha=0.6, width=2)
    nx.draw_networkx_labels(G_complement, pos_comp, ax=axes[1])
    is_bipartite_comp = nx.is_bipartite(G_complement)
    axes[1].set_title(f"$\overline{{K_{2,2}}}$ (Bipartite: {is_bipartite_comp})")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Original is bipartite: {nx.is_bipartite(G)}")
    print(f"Complement is bipartite: {is_bipartite_comp}")

draw_bipartite_with_bipartite_complement()

### (a') Experiment with more examples

In [ ]:
# Experiment with different bipartite graphs
def experiment_bipartite_complements():
    examples = [
        ("K_{2,2}", nx.complete_bipartite_graph(2, 2)),
        ("K_{3,3}", nx.complete_bipartite_graph(3, 3)),
        ("Empty bipartite (3,3)", nx.Graph([(i, j) for i in range(3) for j in range(3, 6)])),
    ]
    
    for name, G in examples:
        G_comp = nx.complement(G)
        print(f"\n{name}:")
        print(f"  Original bipartite: {nx.is_bipartite(G)}")
        print(f"  Complement bipartite: {nx.is_bipartite(G_comp)}")
        print(f"  Original: {G.number_of_nodes()} vertices, {G.number_of_edges()} edges")
        print(f"  Complement: {G_comp.number_of_nodes()} vertices, {G_comp.number_of_edges()} edges")

experiment_bipartite_complements()

### (c) Proposition and Proof

**Proposition:** A bipartite graph $G$ has a bipartite complement if and only if $G = K_{2,2}$ or $G$ is empty (with $n \leq 2$).

**Proof:** 

($\Rightarrow$) If $\overline{G}$ is bipartite, it has no odd cycles. If $G$ is not complete bipartite or has partitions $|X|, |Y| \geq 3$, then $\overline{G}$ contains edges within partitions forming triangles, contradiction. So $|X|, |Y| \leq 2$. For $K_{2,2}$, $\overline{G}$ is two disjoint edges (bipartite). For larger $K_{m,n}$ with $m,n \geq 3$, $\overline{G}$ contains cliques $\geq 3$, not bipartite.

($\Leftarrow$) $K_{2,2}$: complement is two disjoint edges (bipartite). Empty graph: complement is $K_n$, bipartite only for $n \leq 2$. $\square$